# Comprobar que el pc tiene GPU

In [1]:
!nvidia-smi

Mon Dec  9 17:56:13 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA T1000 8GB             WDDM  |   00000000:61:00.0  On |                  N/A |
| 35%   45C    P8             N/A /   50W |    1601MiB /   8192MiB |     40%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Actualizar PIP (Si es necesario)

In [ ]:
%pip install --upgrade pip

# Instalar Dependencias

In [ ]:
%pip install -r requirements.txt

# Importar todas las dependecias

In [31]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from matplotlib import pyplot as plt  # type: ignore
from tensorflow.keras.layers import Dense  # type: ignore
from tensorflow.keras.models import Sequential  # type: ignore
from tensorflow.keras.losses import sparse_categorical_crossentropy  # type: ignore


## Importar los métodos necesarios  

In [32]:
from labeling_images import *

# Creacion de las carpetas 

In [33]:
# Carpeta general
DATA_PATH = os.path.join("D_CORTO")
# Clases
actions = np.array(["manzana", "coco"])
# Número de secuencias
no_sequences = 2
# Secuencia de frames o fotos
sequence_length = 4

In [34]:
print(actions)

['manzana' 'coco']


Se crea la carpeta donde seran guardados los frames de cadad video
    

In [35]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

Captura de los frames, en este caso fotos de la seña que sean importante (4 frames) 

In [36]:
import cv2
import os
import numpy as np
import mediapipe as mp

# Inicializar la captura de video
cap = cv2.VideoCapture(0)

# Variables de configuración
actions = ['coco', 'manzana']  # Lista de frutas
sub_folders = [str(i) for i in range(2)]  # Sub carpetas numeradas de 0 a 1
sequence_length = 4  # Número de fotos por secuencia

# Crear un diccionario para contar las fotos tomadas en cada secuencia
photo_count = {action: {sub_folder: 0 for sub_folder in sub_folders} for action in actions}
action_index = 0  # Índice para la fruta actual
subfolder_index = 0  # Índice para la sub carpeta actual
countdown = sequence_length  # Contador inicial para las fotos

print("Presiona 'd' para tomar una foto. Presiona 'q' para salir.")

with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Hacer las detecciones y dibujar los puntos clave
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)

        # Mostrar el video en una ventana
        cv2.putText(image, f'Fotos restantes: {countdown}', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, f'Carpeta: {actions[action_index]} Sub carpeta: {subfolder_index}', (10, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Video', image)

        # Esperar a que se presione una tecla
        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            break
        elif key == ord('d'):
            # Tomar la foto y guardarla en formato JPG
            current_action = actions[action_index]
            current_subfolder = sub_folders[subfolder_index]
            photo_filename = f'{photo_count[current_action][current_subfolder]}.jpg'
            photo_path = os.path.join(DATA_PATH, current_action, current_subfolder, photo_filename)
            cv2.imwrite(photo_path, image)
            photo_count[current_action][current_subfolder] += 1

            # Extraer y guardar los puntos clave en formato NPY
            keypoints = extract_keypoints(results)
            npy_filename = f'{photo_count[current_action][current_subfolder] - 1}.npy'
            npy_path = os.path.join(DATA_PATH, current_action, current_subfolder, npy_filename)
            np.save(npy_path, keypoints)

            # Actualizar el contador y mostrarlo en pantalla
            countdown -= 1
            cv2.putText(image, f'Foto guardada: {photo_filename}', (10, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.imshow('Video', image)

            if countdown == 0:
                # Cambiar a la siguiente sub carpeta
                subfolder_index += 1
                countdown = sequence_length  # Reiniciar el contador

                if subfolder_index >= len(sub_folders):
                    subfolder_index = 0
                    action_index += 1

                    if action_index >= len(actions):
                        cv2.putText(image, 'Todas las fotos completadas', (10, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
                        cv2.imshow('Video', image)
                        cv2.waitKey(2000)
                        break

cap.release()
cv2.destroyAllWindows()


Presiona 'd' para tomar una foto. Presiona 'q' para salir.


I0000 00:00:1723223959.184038   29540 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723223959.258827   45862 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 560.28.03), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
W0000 00:00:1723223959.300855   45850 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723223959.311228   45851 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723223959.311880   45856 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723223959.312107   45861 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W